In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda, Concatenate

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [4]:
# Custom function to build hidden combination (i.e. convolution) layers

def create_random_combination_layer(input_layer, combination_size, num_combinations, input_dim):
    outputs = []
    
    for _ in range(num_combinations):
        # First random feature selection
        indices_1 = np.random.choice(input_dim, combination_size, replace=False)
        indices_tensor_1 = tf.constant(indices_1, dtype=tf.int32)
        
        # First feature selection using Lambda layer
        slice_layer_1 = Lambda(
            lambda x: tf.gather(x, indices_tensor_1, axis=1),  # Gather selected features
            output_shape=(combination_size,)
        )(input_layer)
        
        # Second random feature selection (after the first random selection)
        indices_2 = np.random.choice(combination_size, combination_size, replace=False)
        indices_tensor_2 = tf.constant(indices_2, dtype=tf.int32)
        
        # Second feature selection using Lambda layer
        slice_layer_2 = Lambda(
            lambda x: tf.gather(x, indices_tensor_2, axis=1),  # Apply a second feature selection
            output_shape=(combination_size,)
        )(slice_layer_1)

        # Apply Dense layers on the final selected subset
        selected_features = Dense(16, activation='relu')(
            Dense(8, activation='relu')(
                Dense(4, activation='relu')(slice_layer_2)
            )
        )
        outputs.append(selected_features)
    
    # Concatenate the outputs from all the random feature combinations
    return Concatenate()(outputs)

In [5]:
def run_model(train_data_path, test_data_path, is_string_labels=False, label_mapping=None):
    # Initialize the one-hot encoder for the target
    encoder = OneHotEncoder(sparse_output=False)

    # Load and Prepare Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if is_string_labels:
        train_data['label'] = train_data['label'].map(label_mapping)
    train_X = train_data.drop(columns=['label']).values
    train_y = train_data['label'].values
    train_y = encoder.fit_transform(train_y.reshape(-1, 1))
    del train_data
    gc.collect()

    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if is_string_labels:
        test_data['label'] = test_data['label'].map(label_mapping)
    test_X = test_data.drop(columns=['label']).values
    test_y = test_data['label'].values
    test_y = encoder.transform(test_y.reshape(-1, 1))
    del test_data
    gc.collect()

    # Parameters for Random Feature Combination
    num_combinations = 20  # Number of random column combinations
    combination_size = 3   # Number of columns in each combination

    # EarlyStopping Callback (optional, to avoid overfitting)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Number of runs for averaging results
    num_runs = 5
    num_folds = 5  # Number of folds for stratified k-fold

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Stratified K-Fold Cross-Validation
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

    for run in range(num_runs):
        fold = 0
        for train_index, val_index in skf.split(train_X, train_y.argmax(axis=1)):  # Assuming one-hot encoding
            fold += 1
            print(f"Run {run + 1}/{num_runs}, Fold {fold}/{num_folds} started...")

            # Create training and validation splits for this fold
            X_train, X_val = train_X[train_index], train_X[val_index]
            y_train, y_val = train_y[train_index], train_y[val_index]

            # Define the model
            input_layer = Input(shape=(X_train.shape[1],))  # Input shape from the training data
            feature_layer = create_random_combination_layer(input_layer, combination_size, num_combinations, X_train.shape[1])
            hidden_layer = Dense(128, activation='relu')(feature_layer)
            hidden_layer = Dropout(0.5)(hidden_layer)
            output_layer = Dense(test_y.shape[1], activation='softmax')(hidden_layer)
            model = Model(inputs=input_layer, outputs=output_layer)
            model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

            # Train the model
            history = model.fit(
                X_train, y_train,
                epochs=1000,
                batch_size=int(X_train.shape[0] * 0.01),
                validation_data=(X_val, y_val),
                callbacks=[early_stopping],
                verbose=0
            )

            # Evaluate on the test set
            test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
            y_pred = model.predict(test_X, verbose=0)
            y_pred_classes = y_pred.argmax(axis=1)
            y_true_classes = test_y.argmax(axis=1)

            # Compute metrics
            balanced_acc = balanced_accuracy_score(y_true_classes, y_pred_classes)
            roc_auc = roc_auc_score(test_y, y_pred, multi_class='ovr')
            pr_auc = average_precision_score(test_y, y_pred, average='weighted')
            f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')
            mcc = matthews_corrcoef(y_true_classes, y_pred_classes)

            # Store metrics
            metrics_storage['test_loss'].append(test_loss)
            metrics_storage['test_accuracy'].append(test_acc)
            metrics_storage['balanced_accuracy'].append(balanced_acc)
            metrics_storage['roc_auc'].append(roc_auc)
            metrics_storage['pr_auc'].append(pr_auc)
            metrics_storage['f2'].append(f2)
            metrics_storage['mcc'].append(mcc)

            # Store classification report metrics
            report = classification_report(y_true_classes, y_pred_classes, output_dict=True)
            for label, values in report.items():
                if isinstance(values, dict):
                    for metric, value in values.items():
                        metrics_storage[f"{label}_{metric}"].append(value)
                else:
                    metrics_storage[label].append(values)

            # Cleanup
            del model
            gc.collect()

    # Average the metrics over all runs and folds
    print("\nAggregated Metrics:")
    for metric, values in metrics_storage.items():
        avg_value = np.mean(values)
        print(f"{metric}: {avg_value:.4f}")

    gc.collect()


In [6]:
run_model("/kaggle/input/ma-datasets/shuttle_train.csv", "/kaggle/input/ma-datasets/shuttle_test.csv", is_string_labels = False)

Run 1/5, Fold 1/5 started...
Run 1/5, Fold 2/5 started...
Run 1/5, Fold 3/5 started...
Run 1/5, Fold 4/5 started...
Run 1/5, Fold 5/5 started...
Run 2/5, Fold 1/5 started...
Run 2/5, Fold 2/5 started...
Run 2/5, Fold 3/5 started...
Run 2/5, Fold 4/5 started...
Run 2/5, Fold 5/5 started...
Run 3/5, Fold 1/5 started...
Run 3/5, Fold 2/5 started...
Run 3/5, Fold 3/5 started...
Run 3/5, Fold 4/5 started...
Run 3/5, Fold 5/5 started...
Run 4/5, Fold 1/5 started...
Run 4/5, Fold 2/5 started...
Run 4/5, Fold 3/5 started...
Run 4/5, Fold 4/5 started...
Run 4/5, Fold 5/5 started...
Run 5/5, Fold 1/5 started...
Run 5/5, Fold 2/5 started...
Run 5/5, Fold 3/5 started...
Run 5/5, Fold 4/5 started...
Run 5/5, Fold 5/5 started...

Aggregated Metrics:
test_loss: 0.2520
test_accuracy: 0.9133
balanced_accuracy: 0.3705
roc_auc: 0.7194
pr_auc: 0.9391
f2: 0.9075
mcc: 0.7349
0_precision: 0.9279
0_recall: 0.9768
0_f1-score: 0.9509
0_support: 9117.0000
1_precision: 0.0356
1_recall: 0.0320
1_f1-score: 0.0337
1

In [7]:
run_model("/kaggle/input/ma-datasets/covtype_train.csv", "/kaggle/input/ma-datasets/covtype_test.csv", is_string_labels = False)

Run 1/5, Fold 1/5 started...
Run 1/5, Fold 2/5 started...
Run 1/5, Fold 3/5 started...
Run 1/5, Fold 4/5 started...
Run 1/5, Fold 5/5 started...
Run 2/5, Fold 1/5 started...
Run 2/5, Fold 2/5 started...
Run 2/5, Fold 3/5 started...
Run 2/5, Fold 4/5 started...
Run 2/5, Fold 5/5 started...
Run 3/5, Fold 1/5 started...
Run 3/5, Fold 2/5 started...
Run 3/5, Fold 3/5 started...
Run 3/5, Fold 4/5 started...
Run 3/5, Fold 5/5 started...
Run 4/5, Fold 1/5 started...
Run 4/5, Fold 2/5 started...
Run 4/5, Fold 3/5 started...
Run 4/5, Fold 4/5 started...
Run 4/5, Fold 5/5 started...
Run 5/5, Fold 1/5 started...
Run 5/5, Fold 2/5 started...
Run 5/5, Fold 3/5 started...
Run 5/5, Fold 4/5 started...
Run 5/5, Fold 5/5 started...

Aggregated Metrics:
test_loss: 1.1340
test_accuracy: 0.5045
balanced_accuracy: 0.1672
roc_auc: 0.6269
pr_auc: 0.4356
f2: 0.4360
mcc: 0.0932
0_precision: 0.3605
0_recall: 0.0862
0_f1-score: 0.1137
0_support: 42368.0000
1_precision: 0.5060
1_recall: 0.9548
1_f1-score: 0.6583


In [8]:
labels_map = {
    'normal.': 0, 'satan.': 1, 'ipsweep.': 2, 'portsweep.': 3, 'nmap.': 4,
    'back.': 5, 'warezclient.': 6, 'teardrop.': 7, 'pod.': 8, 'guess_passwd.': 9,
    'buffer_overflow.': 10, 'land.': 11, 'warezmaster.': 12, 'imap.': 13, 'rootkit.': 14,
    'loadmodule.': 15, 'multihop.': 16, 'ftp_write.': 17, 'phf.': 18, 'perl.': 19, 'spy.': 20
}

run_model("/kaggle/input/ma-datasets/kdd_train.csv", "/kaggle/input/ma-datasets/kdd_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/5, Fold 1/5 started...
Run 1/5, Fold 2/5 started...
Run 1/5, Fold 3/5 started...
Run 1/5, Fold 4/5 started...
Run 1/5, Fold 5/5 started...
Run 2/5, Fold 1/5 started...
Run 2/5, Fold 2/5 started...
Run 2/5, Fold 3/5 started...
Run 2/5, Fold 4/5 started...
Run 2/5, Fold 5/5 started...
Run 3/5, Fold 1/5 started...
Run 3/5, Fold 2/5 started...
Run 3/5, Fold 3/5 started...
Run 3/5, Fold 4/5 started...
Run 3/5, Fold 5/5 started...
Run 4/5, Fold 1/5 started...
Run 4/5, Fold 2/5 started...
Run 4/5, Fold 3/5 started...
Run 4/5, Fold 4/5 started...
Run 4/5, Fold 5/5 started...
Run 5/5, Fold 1/5 started...
Run 5/5, Fold 2/5 started...
Run 5/5, Fold 3/5 started...
Run 5/5, Fold 4/5 started...
Run 5/5, Fold 5/5 started...

Aggregated Metrics:
test_loss: 0.2291
test_accuracy: 0.9575
balanced_accuracy: 0.0647
roc_auc: 0.5813
pr_auc: 0.9293
f2: 0.9494
mcc: 0.1381
0_precision: 0.9588
0_recall: 0.9992
0_f1-score: 0.9785
0_support: 194557.0000
1_precision: 0.1648
1_recall: 0.1226
1_f1-score: 0.1294

In [9]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("/kaggle/input/ma-datasets/darknet_train.csv", "/kaggle/input/ma-datasets/darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/5, Fold 1/5 started...
Run 1/5, Fold 2/5 started...
Run 1/5, Fold 3/5 started...
Run 1/5, Fold 4/5 started...
Run 1/5, Fold 5/5 started...
Run 2/5, Fold 1/5 started...
Run 2/5, Fold 2/5 started...
Run 2/5, Fold 3/5 started...
Run 2/5, Fold 4/5 started...
Run 2/5, Fold 5/5 started...
Run 3/5, Fold 1/5 started...
Run 3/5, Fold 2/5 started...
Run 3/5, Fold 3/5 started...
Run 3/5, Fold 4/5 started...
Run 3/5, Fold 5/5 started...
Run 4/5, Fold 1/5 started...
Run 4/5, Fold 2/5 started...
Run 4/5, Fold 3/5 started...
Run 4/5, Fold 4/5 started...
Run 4/5, Fold 5/5 started...
Run 5/5, Fold 1/5 started...
Run 5/5, Fold 2/5 started...
Run 5/5, Fold 3/5 started...
Run 5/5, Fold 4/5 started...
Run 5/5, Fold 5/5 started...

Aggregated Metrics:
test_loss: 0.5715
test_accuracy: 0.8560
balanced_accuracy: 0.1359
roc_auc: 0.6749
pr_auc: 0.8093
f2: 0.8312
mcc: 0.1391
0_precision: 0.8615
0_recall: 0.9940
0_f1-score: 0.9227
0_support: 26862.0000
1_precision: 0.2190
1_recall: 0.1631
1_f1-score: 0.1815
